In [10]:
import numpy as np
from math import log
import operator

def createDataSet():
    group = [[1,1,'A'], [1,1,'A'], [1,0,'B'], [0,1,'B'],[0,1,'B']]
    return group

# 计算信息熵
def calcShannonEnt(dataSet):
    count = len(dataSet)
    labelsCount = {}
    for featVec in dataSet:
        feat = featVec[-1]
        if feat not in labelsCount.keys():
            labelsCount[feat] = 0
        labelsCount[feat] += 1
    result = 0.0
    for key in labelsCount:
        prop = float(labelsCount[key] / count)
        result += (-prop)*log(prop,2)
    return result

group = createDataSet()
#shannonEnt = calcShannonEnt(group)
#print(shannonEnt)

def splitDataSet(dataSet, index, value):
    result = []
    for featVec in dataSet:
        if featVec[index] == value:
            tmpVec = featVec[:index]
            tmpVec.extend(featVec[index+1:])
            result.append(tmpVec)
    return result

#splitDataSet = splitDataSet(group, 1, 1.0)
#print(splitDataSet)

def bestFeatureToSplit(dataSet):
    m = len(dataSet)
    k = len(dataSet[0]) - 1
    shannonEnt = calcShannonEnt(dataSet)
    bestInfoGain = 0
    bestIndex = -1
    for i in range(k):
        featList = [item[i] for item in dataSet]
        featSet = set(featList)
        splitShannonEnt = 0
        for feat in featSet:
            subDataSet = splitDataSet(dataSet,i,feat)
            prop = len(subDataSet) / float(m)
            splitShannonEnt += prop * calcShannonEnt(subDataSet)
            infoGain = shannonEnt - splitShannonEnt
            if infoGain > bestInfoGain:
                bestInfoGain = infoGain
                bestIndex = i
    return bestIndex

#bestIndex = bestFeatureToSplit(group)
#print(bestIndex)

def majorityCnt(dataList):
    labelCountDic = {}
    for item in dataList:
        if item not in labelCountDic.keys():
            labelCountDic[item] = 0
        labelCountDic[item] += 1
    sortedLabelCount = sorted(labelCountDic.items(), key = operator.itemgetter(1), reverse = True)
    return sortedLabelCount[0][0]

def createDecisionTree(dataSet, labels):
    labelList = [item[-1] for item in dataSet]
    if len(dataSet[0]) == 1:
        return majorityCnt(labelList)
    if labelList.count(labelList[0]) == len(labelList):
        return labelList[0]
    bestIndex = bestFeatureToSplit(group)
    bestLabel = labels[bestIndex]
    tree = {bestLabel:{}}
    del(labels[bestIndex])
    subValues = [item[bestIndex] for item in dataSet]
    uniqueValues = set(subValues)
    for subValue in subValues:
        subDataSet = splitDataSet(dataSet, bestIndex, subValue)
        tree[bestLabel][subValue] = createDecisionTree(subDataSet, labels[:])
    return tree

tree = createDecisionTree(group,['a', 'b'])
print(tree)

{'a': {1: {'b': {1: 'A', 0: 'B'}}, 0: 'B'}}
